In [1]:
# -----------------------------------------------
# 
# postprocessing for the basalt and 
# wollastonite overflow results
# 
# -----------------------------------------------

# --- Decide which CDR calculations to perform
cdr_calc_list = ["co2_flx",       
                 "camg_flx",
                 "totcat_flx",
                 # "carbalk_flx", # (turn this back on when the runs are re-run with fixed cflx file)
                 "rockdiss"]
# ---

In [6]:
import fnmatch
import glob
import os
import pickle
import re
from typing import Tuple

import cmocean.cm as cmo
import fsspec         # for AWS integration
import s3fs
from matplotlib.cm import ScalarMappable
from matplotlib.colors import TwoSlopeNorm
from matplotlib.colors import Normalize
from matplotlib.lines import Line2D  # for custom legend entries (needed for contour plot)
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import cdr_fxns_postproc as cfp


# --- 
runtype = "field"   # field or lab
fertlevel = "no"   # "no", "low", "mid", or "hi"
dustsp1 = "gbas"     # "gbas", "wls" (silicate species)
dustsp2 = "wls"
sitename = "311a"   # "311a" or "311b"
multiyear_fs1 = False    # whether we are looking for multi-year composites for silicate case
multiyear_fs2 = False   # whether we are looking for multi-year composites for cc case
dur = '200'           # [yr], for file naming

tag_fs1 = "overflow_v1"  # an extra tag in the casenames, something like "wet" (use "" for none)
tag_fs2 = "overflow"   # an extra tag in the casenames, something like "wet" (use "" for none)

version_fs1_csv = "v2"   # version appended to the csv file for the batch input
version_fs2_csv = "v0"

# --- SAVE DETAILS -----------------------------------------------------------------------------------------
save_results = True
if multiyear_fs1 or multiyear_fs2:
    savedir_pref = f"meanAnn_shortRun_{fertlevel}Fert_{tag_fs1}-{tag_fs2}_multiyear"   # prefix for the save directory (which will be created upon save)
else:
    savedir_pref = f"meanAnn_shortRun_{fertlevel}Fert_{tag_fs1}-{tag_fs2}"   # prefix for the save directory (which will be created upon save)

savepath = "/home/tykukla/aglime-swap-cdr/scepter/process/runs/batch_postprocResults/cc-sil_psize_apprate"
# ----------------------------------------------------------------------------------------------------------


# [FEEDSTOCK 1 RUNS]
if multiyear_fs1:
    runname_ctrl_fs1 = [f"{fertlevel}Fert_{dustsp1}_multiyear_site_{sitename}_app_0p0_psize_*_composite_{runtype}"]
    csv_fn_fs1 = f"meanAnn_{dustsp1}_shortRun_multiyear_{fertlevel}Fert_gs+apprate_{version_fs1_csv}.csv"
else:
    runname_ctrl_fs1 = [f"{fertlevel}Fert_{dustsp1}_{tag_fs1}_site_{sitename}_app_0p0_psize_*_{dustsp1}_{runtype}_tau{dur}p0"]
    csv_fn_fs1 = f"meanAnn_{dustsp1}_shortRun_{tag_fs1}_{fertlevel}Fert_gs+apprate_{version_fs1_csv}.csv"

# [FEEDSTOCK 2 RUNS]
if multiyear_fs2:
    runname_ctrl_fs2 = [f"{fertlevel}Fert_{dustsp2}_multiyear_site_{sitename}_app_0p0_psize_*_composite_{runtype}"]
    csv_fn_fs2 = f"meanAnn_{dustsp2}_shortRun_multiyear_{fertlevel}Fert_gs+apprate_{version_fs2_csv}.csv"
else:
    runname_ctrl_fs2 = [f"{fertlevel}Fert_{dustsp2}_{tag_fs2}_site_{sitename}_app_0p0_psize_*_{dustsp2}_{runtype}_tau{dur}p0"]
    csv_fn_fs2 = f"meanAnn_{dustsp2}_shortRun_{tag_fs2}_{fertlevel}Fert_gs+apprate_{version_fs2_csv}.csv"

# ---
# groundwork
# outdir = "/home/tykukla/SCEPTER/scepter_output"
outdir = "s3://carbonplan-carbon-removal/SCEPTER/scepter_output_scratch/"
csv_loc = "/home/tykukla/aglime-swap-cdr/scepter/batch-inputs"



In [7]:
# --- read in the batch .csv 
dfin_fs1 = pd.read_csv(os.path.join(csv_loc, csv_fn_fs1))
dfin_fs2 = pd.read_csv(os.path.join(csv_loc, csv_fn_fs2))

# [FEEDSTOCK 1]
# add column for the full run id
if multiyear_fs1:
    dfin_fs1["newrun_id_full"] = dfin_fs1['newrun_id'] + f"_composite_{runtype}"
else:
    dfin_fs1["newrun_id_full"] = dfin_fs1['newrun_id'] + "_" + dfin_fs1['dustsp'] + "_" + runtype + "_tau"+dfin_fs1["duration"].astype(float).astype(str).str.replace(".", "p")  # (duration has to be turned into float first because otherwise we miss the decimal pt)
# identify the control run
dfin_fs1["ctrl_run"] = dfin_fs1['newrun_id_full'].apply(lambda filename: any(fnmatch.fnmatch(filename, pattern) for pattern in runname_ctrl_fs1))
# eliminate other sites if necessary
if len(dfin_fs1['site'].unique()) > 1:
    dfin_fs1 = dfin_fs1[dfin_fs1['site'] == f"site_{sitename}"].copy()
# add a column for the dustrate in ton_ha_yr
if "dustrate" in dfin_fs1.columns:
    dfin_fs1["dustrate_ton_ha_yr"] = dfin_fs1["dustrate"] / 100 
    
# [FEEDSTOCK 2]
# add column for the full run id
if multiyear_fs2:
    dfin_fs2["newrun_id_full"] = dfin_fs2['newrun_id'] + f"_composite_{runtype}"
else:
    dfin_fs2["newrun_id_full"] = dfin_fs2['newrun_id'] + "_" + dfin_fs2['dustsp'] + "_" + runtype + "_tau"+dfin_fs2["duration"].astype(float).astype(str).str.replace(".", "p")  # (duration has to be turned into float first because otherwise we miss the decimal pt)
# identify the control run
dfin_fs2["ctrl_run"] = dfin_fs2['newrun_id_full'].apply(lambda filename: any(fnmatch.fnmatch(filename, pattern) for pattern in runname_ctrl_fs2))
# eliminate other sites if necessary
if len(dfin_fs2['site'].unique()) > 1:
    dfin_fs2 = dfin_fs2[dfin_fs2['site'] == f"site_{sitename}"].copy()
# add a column for the dustrate in ton_ha_yr
if "dustrate" in dfin_fs2.columns:
    dfin_fs2["dustrate_ton_ha_yr"] = dfin_fs2["dustrate"] / 100 


In [8]:
dfin_fs2

,dustrate,dustrad,site,spinrun,climatefiles,duration,dustsp,dustsp_2nd,dustrate_2nd,add_secondary,...,poro_iter_field,poro_evol,cec_adsorption_on,climatedir,aws_save,aws_bucket,newrun_id,newrun_id_full,ctrl_run,dustrate_ton_ha_yr
0,0.0,25,site_311a,site_311a_pr7_spintuneup4,site_311a,200,wls,amnt,0.0,True,...,True,True,True,NaN,move,s3://carbonplan-carbon-removal/SCEPTER/scepter...,noFert_wls_overflow_site_311a_app_0p0_psize_25,noFert_wls_overflow_site_311a_app_0p0_psize_25...,True,0.0
1,200.0,25,site_311a,site_311a_pr7_spintuneup4,site_311a,200,wls,amnt,0.0,True,...,True,True,True,NaN,move,s3://carbonplan-carbon-removal/SCEPTER/scepter...,noFert_wls_overflow_site_311a_app_200p0_psize_25,noFert_wls_overflow_site_311a_app_200p0_psize_...,False,2.0
2,200.0,50,site_311a,site_311a_pr7_spintuneup4,site_311a,200,wls,amnt,0.0,True,...,True,True,True,NaN,move,s3://carbonplan-carbon-removal/SCEPTER/scepter...,noFert_wls_overflow_site_311a_app_200p0_psize_50,noFert_wls_overflow_site_311a_app_200p0_psize_...,False,2.0
3,1000.0,25,site_311a,site_311a_pr7_spintuneup4,site_311a,200,wls,amnt,0.0,True,...,True,True,True,NaN,move,s3://carbonplan-carbon-removal/SCEPTER/scepter...,noFert_wls_overflow_site_311a_app_1000p0_psize_25,noFert_wls_overflow_site_311a_app_1000p0_psize...,False,10.0
4,1000.0,50,site_311a,site_311a_pr7_spintuneup4,site_311a,200,wls,amnt,0.0,True,...,True,True,True,NaN,move,s3://carbonplan-carbon-removal/SCEPTER/scepter...,noFert_wls_overflow_site_311a_app_1000p0_psize_50,noFert_wls_overflow_site_311a_app_1000p0_psize...,False,10.0
5,4000.0,25,site_311a,site_311a_pr7_spintuneup4,site_311a,200,wls,amnt,0.0,True,...,True,True,True,NaN,move,s3://carbonplan-carbon-removal/SCEPTER/scepter...,noFert_wls_overflow_site_311a_app_4000p0_psize_25,noFert_wls_overflow_site_311a_app_4000p0_psize...,False,40.0
6,4000.0,50,site_311a,site_311a_pr7_spintuneup4,site_311a,200,wls,amnt,0.0,True,...,True,True,True,NaN,move,s3://carbonplan-carbon-removal/SCEPTER/scepter...,noFert_wls_overflow_site_311a_app_4000p0_psize_50,noFert_wls_overflow_site_311a_app_4000p0_psize...,False,40.0
7,6000.0,25,site_311a,site_311a_pr7_spintuneup4,site_311a,200,wls,amnt,0.0,True,...,True,True,True,NaN,move,s3://carbonplan-carbon-removal/SCEPTER/scepter...,noFert_wls_overflow_site_311a_app_6000p0_psize_25,noFert_wls_overflow_site_311a_app_6000p0_psize...,False,60.0
8,6000.0,50,site_311a,site_311a_pr7_spintuneup4,site_311a,200,wls,amnt,0.0,True,...,True,True,True,NaN,move,s3://carbonplan-carbon-removal/SCEPTER/scepter...,noFert_wls_overflow_site_311a_app_6000p0_psize_50,noFert_wls_overflow_site_311a_app_6000p0_psize...,False,60.0


In [9]:
# --- decide which columns from dfin we want to keep when we 
#     create our flux dicts below
# 
# these should be the columns that become dimensions in the 
# later xr datasets... 
# 
dfin_cols_to_keep = ["dustrad", "dustrate_ton_ha_yr"]

In [10]:
# --- read in the postprocessed flux data for fs1
flx_dict_fs1 = cfp.read_postproc_flux(dfin_fs1, outdir, cdr_calc_list, dfin_cols_to_keep, rockdiss_feedstock=dustsp1)

solving co2_flx
solving camg_flx
solving totcat_flx
solving rockdiss


In [11]:
# --- read in the postprocessed flux data for fs2
# (TK skip wls for now... not done running?)
# flx_dict_fs2 = cfp.read_postproc_flux(dfin_fs2, outdir, cdr_calc_list, dfin_cols_to_keep, rockdiss_feedstock=dustsp2)

## More to come with the cdr postprocessing later ...
Right now I am only computing integrated CDR... I need to get time-evolution of integrated CDR...

# PROFILE POSTPROCESSING

In [12]:
# --- prepare the post-processing inputs
# 
batch_axes = dfin_cols_to_keep
#
# dictionary for which files to process in the batch profile functions
proc_dict = {
    "adsorbed": False,
    "adsorbed_percCEC": True,
    "adsorbed_ppm": False,
    "aqueous": True,
    "aqueous_total": False,
    "bulksoil": True,
    "exchange_total": False,
    "gas": True,
    "rate": False,
    "soil_ph": True,
    "solid": False,
    "solid_sp_saturation": True, 
    "solid_volumePercent": True,
    "solid_weightPercent": True,
    "specific_surface_area": True,
    "surface_area": False,
}


In [13]:
# --- compile all the profile .nc data into a single dataset

# [FEEDSTOCK 1]
dsdict_fs1 = cfp.prof_batchprocess_allvars(outdir, dustsp1, dfin_fs1, batch_axes, proc_dict, print_loop_updates = False)

compiling profile data for gbas


In [15]:
# --- compile all the profile .nc data into a single dataset

# [FEEDSTOCK 2]
# dsdict_fs2 = cfp.prof_batchprocess_allvars(outdir, dustsp2, dfin_fs2, batch_axes, proc_dict, print_loop_updates = False)

In [14]:
# --- save the postprocessed results
# 

# [FEEDSTOCK 1]
cfp.save_batch_postproc_profOnly(dsdict_fs1, dustsp1, base_path=savepath, base_dir_name=savedir_pref)
# cfp.save_batch_postproc_profOnly(dsdict_fs1, dustsp1, save_directory = None)

# [FEESTOCK 2]
# cfp.save_batch_postproc_profOnly(dsdict_fs2, dustsp2, base_path=savepath, base_dir_name=savedir_pref)
# cfp.save_batch_postproc_profOnly(dsdict_fs2, dustsp2, save_directory = saved_here)

Directory created: /home/tykukla/aglime-swap-cdr/scepter/process/runs/batch_postprocResults/cc-sil_psize_apprate/meanAnn_shortRun_noFert_overflow_v1-overflow_001


'/home/tykukla/aglime-swap-cdr/scepter/process/runs/batch_postprocResults/cc-sil_psize_apprate/meanAnn_shortRun_noFert_overflow_v1-overflow_001'